<a href="https://colab.research.google.com/github/PravinTiwari023/SparkNLP-Project/blob/main/SparkNLPProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install spark-nlp==5.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.3/547.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
# !pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=f705573ffd9c01f037a01e180f4b4815f1105c008f2673aaee82238a832df2b9
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [1]:
# Ignore this part
import pandas as pd
import numpy as np
import json

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [2]:
# Ignore this part
spark = sparknlp.start()
print ("Spark NLP Version :", sparknlp.version())
spark

Spark NLP Version : 5.2.2


In [3]:
# Ignore this part
document_assembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("documents")

t5_ = T5Transformer() \
  .pretrained("t5_small", 'en') \
  .setTask("summarize:")\
  .setMaxOutputLength(200)\
  .setInputCols(["documents"]) \
  .setOutputCol("summaries")

summarizer_pp = Pipeline(stages=[document_assembler, t5_])

t5_small download started this may take some time.
Approximate size to download 241.9 MB
[OK!]


In [4]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

t5 = T5Transformer.pretrained("t5_passive_to_active_styletransfer") \
    .setTask("transfer Passive to Active:") \
    .setInputCols(["documents"]) \
    .setMaxOutputLength(200) \
    .setOutputCol("transfers")

pipeline = Pipeline().setStages([documentAssembler, t5])

t5_passive_to_active_styletransfer download started this may take some time.
Approximate size to download 253.2 MB
[OK!]


In [7]:
from typing_extensions import Text
class NLPApp:
  # Constructor
  def __init__(self):
    self.__database = {}
    self.__First_menu()

  def __First_menu(self):
    first_input = input('''Hi! How would you like to proceed?
    1. Register
    2. Login
    3. Exit\n''')

    if first_input == '1':
      # SignUp
      self.__Register()
      pass

    elif first_input == '2':
      # SignIn
      self.__Login()
      pass

    else:
      # Exit
      print('Thank You')
      exit()

  def __Second_menu(self):
    first_input = input('''Hi! How would you like to proceed?
    1. Text Summarization
    2. Passive to Active
    3. Exit\n''')

    if first_input == '1':
      # SignUp
      self.__Text_summary()
      pass

    elif first_input == '2':
      # SignIn
      self.__Active()
      pass

    else:
      # Exit
      print('Thank You')
      exit()

  def __Register(self):
    name = input('Name: ')
    email = input('Email: ')
    password = input('Password: ')
    cpassword = input('Confirm Password: ')

    if cpassword == password:
      if email is self.__database:
        print('\nEmail is already registered. Back to Login!')
        self.__Login()
      else:
        self.__database[email] = [name, password]
        print('\nRegistration Successful. Now Login!')
        self.__Login()

    else:
      print('\nPassword are not matching')
      self.__Register()

  def __Login(self):
    email = input('Email: ')
    password = input('Password: ')

    if email in self.__database:
      if self.__database[email][1] == password:
        print('\nLogin Successful')
        self.__Second_menu()
    else:
      print('\nThis email is not registered')
      self.__First_menu()

  def __Text_summary(self):
    text = input('Enter Text: ')
    empty_df = spark.createDataFrame([['']]).toDF('text')
    pipeline_model = summarizer_pp.fit(empty_df)
    sum_lmodel = LightPipeline(pipeline_model)
    res = sum_lmodel.fullAnnotate(text)[0]
    print('Summary:', res['summaries'][0].result)
    self.__Second_menu()

  def __Active(self):
    text = input('Enter Text: ')
    pipeline_model = pipeline.fit(spark.createDataFrame([['']]).toDF('text'))
    T5model = LightPipeline(pipeline_model)
    res = T5model.fullAnnotate(text)[0]
    print ('Prediction:', res['transfers'][0].result)
    self.__Second_menu()

# Creating class object
App = NLPApp()

Hi! How would you like to proceed?
    1. Register
    2. Login
    3. Exit
1
Name: Pravin Tiwart
Email: pravin@gmail.com
Password: 1234
Confirm Password: 1234

Registration Successful. Now Login!
Email: pravin@gmail.com
Password: 1234

Login Successful
Hi! How would you like to proceed?
    1. Text Summarization
    2. Passive to Active
    3. Exit
1
Enter Text: There are times when the night sky glows with bands of color. The bands may begin as cloud shapes and then spread into a great arc across the entire sky. They may fall in folds like a curtain drawn across the heavens. The lights usually grow brighter, then suddenly dim. During this time the sky glows with pale yellow, pink, green, violet, blue, and red. These lights are called the Aurora Borealis. Some people call them the Northern Lights. Scientists have been watching them for hundreds of years. They are not quite sure what causes them. In ancient times people were afraid of the Lights. They imagined that they saw fiery drago